# Spectral indices--Sentinel-2l2a

In [1]:
import numpy as np
import rasterio
from pathlib import Path

In [2]:
# Set paths
input_dir = Path("G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked")
output_dir = Path("G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices")

In [3]:
def calculate_ndvi(nir, red):
    """Calculate NDVI: (NIR - RED)/(NIR + RED)"""
    return (nir - red) / (nir + red + 1e-10)

def calculate_ndwi(green, nir):
    """Calculate NDWI: (GREEN - NIR)/(GREEN + NIR)"""
    return (green - nir) / (green + nir + 1e-10)

def calculate_savi(nir, red, L=0.5):
    """Calculate SAVI: ((NIR - RED)/(NIR + RED + L)) * (1 + L)"""
    return ((nir - red) / (nir + red + L + 1e-10)) * (1 + L)

## Process spectral indices for a specific scene

In [8]:
def process_indices_for_scene(scene_dir, output_dir):
    date_str = scene_dir.name
    indices_dir = output_dir / date_str
    indices_dir.mkdir(parents=True, exist_ok=True)

    try:
        nir_path = scene_dir / "B08.tif"
        red_path = scene_dir / "B04.tif"
        green_path = scene_dir / "B03.tif"

        if not nir_path.exists() or not red_path.exists() or not green_path.exists():
            print(f"Skipping {date_str} due to missing bands.")
            return None

        with rasterio.open(nir_path) as nir_src:
            nir = nir_src.read(1).astype(np.float32) / 10000
            profile = nir_src.profile.copy()

        with rasterio.open(red_path) as red_src:
            red = red_src.read(1).astype(np.float32) / 10000

        with rasterio.open(green_path) as green_src:
            green = green_src.read(1).astype(np.float32) / 10000

        # Calculate indices
        ndvi = calculate_ndvi(nir, red)
        ndwi = calculate_ndwi(green, nir)
        savi = calculate_savi(nir, red)

        # Update profile for saving
        profile.update(dtype=rasterio.float32, nodata=-9999)

        # Save indices
        indices = {'ndvi': ndvi, 'ndwi': ndwi, 'savi': savi}
        for name, data in indices.items():
            output_path = indices_dir / f"{name}.tif"
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(data, 1)

        print(f"Successfully processed indices for {date_str}")
        return indices_dir
    except Exception as e:
        print(f"Error processing indices for {date_str}: {str(e)}")
        return None

## Process spectral indices for all available scenes

In [11]:
def process_all_scenes_for_indices(input_dir, output_dir):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    processed_dirs = []
    for scene_dir in input_dir.iterdir():
        if scene_dir.is_dir():
            result = process_indices_for_scene(scene_dir, output_dir)
            if result:
                processed_dirs.append(result)
    return processed_dirs

In [13]:
# Run processing
process_all_scenes_for_indices(input_dir, output_dir)

Skipping S2A_MSIL2A_20240531T080611_R078_T35MQU_20240531T150446 due to missing bands.
Skipping S2A_MSIL2A_20240610T080611_R078_T35MRU_20240610T142004 due to missing bands.
Skipping S2A_MSIL2A_20240620T080611_R078_T35MQU_20240620T135853 due to missing bands.
Skipping S2A_MSIL2A_20240620T080611_R078_T35MRU_20240620T135115 due to missing bands.
Skipping S2A_MSIL2A_20240630T080611_R078_T35MQU_20240701T185708 due to missing bands.
Skipping S2A_MSIL2A_20240630T080611_R078_T35MRU_20240701T190110 due to missing bands.
Skipping S2A_MSIL2A_20240630T080611_R078_T35MRU_20240701T190926 due to missing bands.
Successfully processed indices for S2A_MSIL2A_20240819T080611_R078_T35MQU_20240819T130750
Skipping S2B_MSIL2A_20240127T081119_R078_T35MRU_20240127T120827 due to missing bands.
Skipping S2B_MSIL2A_20240307T080759_R078_T35MRU_20240307T123828 due to missing bands.
Skipping S2B_MSIL2A_20240317T080649_R078_T35MQU_20240317T121450 due to missing bands.
Skipping S2B_MSIL2A_20240317T080649_R078_T35MRU_20

[WindowsPath('G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices/S2A_MSIL2A_20240819T080611_R078_T35MQU_20240819T130750'),
 WindowsPath('G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices/S2B_MSIL2A_20240705T080609_R078_T35MRU_20240706T024811'),
 WindowsPath('G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices/S2B_MSIL2A_20240715T080609_R078_T35MRU_20240715T131113'),
 WindowsPath('G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices/S2B_MSIL2A_20240725T080609_R078_T35MQU_20240725T102837'),
 WindowsPath('G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices/S2B_MSIL2A_20240725T080609_R078_T35MRU_20240725T102837'),
 WindowsPath('G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices/S2B_MSIL2A_20240804T080609_R078_T35MQU_20240804T111459'),
 WindowsPath('G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices/S2B_MSIL2A_20240804T080609_R078_T35MRU_20240804T111459'),
 WindowsPath('G:/Semester4/Innolab/eoAPI/data/sentinel2_l2a_indices/S2B_MSIL2A_20240903T080609_R078_T35MQU_20240903T11